<a href="https://colab.research.google.com/github/onehungrybird/Agentic_AI_travel_planner/blob/main/autonomous_research_agent5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install faiss-cpu
!pip install langchain_community
!pip install -U transformers accelerate bitsandbytes --quiet
!pip install -U bitsandbytes
!pip install chromadb pypdf arxiv

In [3]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map='auto'
)

# Set pad token if not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Function to generate responses
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        pad_token_id=tokenizer.eos_token_id
    )
    # Decode the entire batch at once (outputs is already a tensor)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# Test the model
print(generate_response("Explain the importance of data augmentation in deep learning."))

Explain the importance of data augmentation in deep learning.

Data augmentation is a crucial technique used in deep learning to artificially increase the amount of data available for training. It is important for several reasons:

1. Improving model performance: By augmenting the data, deep learning models can be trained on a larger and more diverse set of data, which can lead to improved performance. This is because the model learns to recognize patterns and features in the data more effectively, leading to better accuracy and generalization.

2. Reducing overfitting: Overfitting occurs when a model becomes too specialized to the training data and is unable to generalize to new data. Data augmentation can help reduce overfitting by providing the model with a more diverse set of training data, which forces the model to learn more robust features.

3. Increasing training efficiency: With data augmentation, deep learning models can be trained on a larger amount of data in the same amoun

In [2]:
import arxiv

def fetch_papers(query, max_results=3):
    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate
    )
    papers = []
    for result in search.results():
        papers.append({
            "title": result.title,
            "summary": result.summary,
            "url": result.entry_id
        })
    return papers

In [3]:
# Example usage
papers = fetch_papers("deep learning for EEG")
for paper in papers:
    print(f"Title: {paper['title']}\nSummary: {paper['summary']}\nURL: {paper['url']}\n")

<ipython-input-2-dd5836b357cf>:10: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


Title: Q-Insight: Understanding Image Quality via Visual Reinforcement Learning
Summary: Image quality assessment (IQA) focuses on the perceptual visual quality of
images, playing a crucial role in downstream tasks such as image
reconstruction, compression, and generation. The rapid advancement of
multi-modal large language models (MLLMs) has significantly broadened the scope
of IQA, moving toward comprehensive image quality understanding that
incorporates content analysis, degradation perception, and comparison reasoning
beyond mere numerical scoring. Previous MLLM-based methods typically either
generate numerical scores lacking interpretability or heavily rely on
supervised fine-tuning (SFT) using large-scale annotated datasets to provide
descriptive assessments, limiting their flexibility and applicability. In this
paper, we propose Q-Insight, a reinforcement learning-based model built upon
group relative policy optimization (GRPO), which demonstrates strong visual
reasoning capabil

In [6]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from transformers import pipeline

# Create Hugging Face pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,
    temperature=0.7
)

# Wrap in LangChain
llm = HuggingFacePipeline(pipeline=pipe)

# Define prompt template
summary_prompt = PromptTemplate(
    input_variables=["paper_text"],
    template="Summarize this research paper in 3 bullet points: {paper_text}"
)

# Create chain
summary_chain = LLMChain(llm=llm, prompt=summary_prompt)

# Run summarization
summary = summary_chain.run({"paper_text": papers[0]['summary']})
print(summary)

Device set to use cuda:0
<ipython-input-6-2e51c8df3515>:16: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)
<ipython-input-6-2e51c8df3515>:28: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary = summary_chain.run({"paper_text": papers[0]['summary']})
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or u

Summarize this research paper in 3 bullet points: Image quality assessment (IQA) focuses on the perceptual visual quality of
images, playing a crucial role in downstream tasks such as image
reconstruction, compression, and generation. The rapid advancement of
multi-modal large language models (MLLMs) has significantly broadened the scope
of IQA, moving toward comprehensive image quality understanding that
incorporates content analysis, degradation perception, and comparison reasoning
beyond mere numerical scoring. Previous MLLM-based methods typically either
generate numerical scores lacking interpretability or heavily rely on
supervised fine-tuning (SFT) using large-scale annotated datasets to provide
descriptive assessments, limiting their flexibility and applicability. In this
paper, we propose Q-Insight, a reinforcement learning-based model built upon
group relative policy optimization (GRPO), which demonstrates strong visual
reasoning capability for image quality understanding whi

In [7]:
hypothesis_prompt = PromptTemplate(
    input_variables=["summary"],
    template="Based on this summary, suggest two new research hypotheses: {summary}"
)

hypothesis_chain = LLMChain(llm=llm, prompt=hypothesis_prompt)
hypothesis = hypothesis_chain.run({"summary": summary})

print("\nProposed Hypotheses:\n", hypothesis)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



Proposed Hypotheses:
 Based on this summary, suggest two new research hypotheses: Summarize this research paper in 3 bullet points: Image quality assessment (IQA) focuses on the perceptual visual quality of
images, playing a crucial role in downstream tasks such as image
reconstruction, compression, and generation. The rapid advancement of
multi-modal large language models (MLLMs) has significantly broadened the scope
of IQA, moving toward comprehensive image quality understanding that
incorporates content analysis, degradation perception, and comparison reasoning
beyond mere numerical scoring. Previous MLLM-based methods typically either
generate numerical scores lacking interpretability or heavily rely on
supervised fine-tuning (SFT) using large-scale annotated datasets to provide
descriptive assessments, limiting their flexibility and applicability. In this
paper, we propose Q-Insight, a reinforcement learning-based model built upon
group relative policy optimization (GRPO), which 

In [11]:
report_prompt = PromptTemplate(
    input_variables=["summary", "hypothesis"],
    template="Research Report\n\nSummary:\n{summary}\n\nHypothesis:\n{hypothesis}\n\nExperiment Results:\n\nConclusion:\nWrite a conclusion based on these findings."
)

report_chain = LLMChain(llm=llm, prompt=report_prompt)
report = report_chain.run({"summary": summary, "hypothesis": hypothesis})

print("\nFinal Research Report:\n", report)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



Final Research Report:
 Research Report

Summary:
Summarize this research paper in 3 bullet points: Image quality assessment (IQA) focuses on the perceptual visual quality of
images, playing a crucial role in downstream tasks such as image
reconstruction, compression, and generation. The rapid advancement of
multi-modal large language models (MLLMs) has significantly broadened the scope
of IQA, moving toward comprehensive image quality understanding that
incorporates content analysis, degradation perception, and comparison reasoning
beyond mere numerical scoring. Previous MLLM-based methods typically either
generate numerical scores lacking interpretability or heavily rely on
supervised fine-tuning (SFT) using large-scale annotated datasets to provide
descriptive assessments, limiting their flexibility and applicability. In this
paper, we propose Q-Insight, a reinforcement learning-based model built upon
group relative policy optimization (GRPO), which demonstrates strong visual
reaso